# Creating the First Commitment Transactions

In this section we'll create the First Commitment transactions of a Lightning Channel from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and the message the peers exchange to send and get the needed information from each other to make it happen. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'

## Commitment Transactions

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md))  details the exact format of on-chain transactions, which both sides need to agree on to ensure signatures are valid. This consists of the funding transaction output script, the commitment transactions, and the HTLC transactions.

This notebook will focus on commitment transactions.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'

Alice send the `open_channel` message to Bob. This message define many channel operational criterias, here we are going to focus on the information that are used in the commitment transaction:
* push_msat
* dust_limit_satoshis
* feerate_per_kw
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

If Bob does not agree with the criterias sent into the `open_channel` message he send a [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice, otherwise he sends the [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, containing some operational criterias from your side, here we are going to focus on the information that are used in the commitment transaction:
* dust_limit_satoshis
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

## Setup
### Requirements

For this exercise we'll run the previous notebook to create the channel funding transaction beetween Alice and Bob.

**You'll need to edit these next line for your local setup.**

In [5]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 1 - Funding Transactions/Funding-Transaction.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

## The First Per Commitment Point
The I'th per commitment secret must match the output of this algorithm:
```
generate_from_seed(seed, I):
    P = seed
    for B in 47 down to 0:
        if B set in I:
            flip(B) in P
            P = SHA256(P)
    return P
```
Where "flip(B)" alternates the (B mod 8) bit of the (B div 8) byte of the value. The first secret used must be index 281474976710655 (0xFFFFFFFFFFFF), and from there, the index is decremented.

The per_commitment_point is generated using elliptic-curve multiplication:

```
per_commitment_point = per_commitment_secret * G
```

In [26]:
import hashlib
# Flip the (bit_index % 8) bit of the (bit_index // 8) byte in value.
def flip_bit(value: bytearray, bit_index: int):
    byte_index = bit_index // 8  # Get the byte index
    bit_position = bit_index % 8  # Get the bit position within the byte
    value[byte_index] ^= (1 << bit_position)  # Flip the bit using XOR

# Generate a derived value P from a seed and an integer I
# param seed: 32-byte initial seed.
# param I: Index whose set bits determine which bits in P get flipped.
# return: 32-byte derived value P.
def generate_from_seed(seed: bytes, I: int) -> bytes:
    P = bytearray(seed)  # Convert to mutable bytearray

    for B in range(47, -1, -1):  # Iterate from 47 down to 0
        if (I & (1 << B)) != 0:  # Check if bit B is set in I
            flip_bit(P, B)  # Flip the corresponding bit in P
            P = bytearray(hashlib.sha256(P).digest())  # Hash P and convert back to bytearray

    return bytes(P)  # Convert back to bytes before returning

alice_first_per_commitment_secret = generate_from_seed(alice_per_commitment_seed, 281474976710655)
alice_first_per_commitment_point_uncompressed = int.from_bytes(alice_first_per_commitment_secret, byteorder="big") * G
alice_first_per_commitment_point = compress_pubkey(alice_first_per_commitment_point_uncompressed)

print(f"Alice First Per Commitment Secret: {alice_first_per_commitment_secret.hex()}")
print(f"Alice First Per Commitment Point: {alice_first_per_commitment_point}")


bob_first_per_commitment_secret = generate_from_seed(bob_per_commitment_seed, 281474976710655)
bob_first_per_commitment_point_uncompressed = int.from_bytes(bob_first_per_commitment_secret, byteorder="big") * G
bob_first_per_commitment_point = compress_pubkey(bob_first_per_commitment_point_uncompressed)

print(f"Bob First Per Commitment Secret: {bob_first_per_commitment_secret.hex()}")
print(f"Bob First Per Commitment Point: {bob_first_per_commitment_point}")


Alice First Per Commitment Secret: 739d879e3b2502146546b9d6d51b1e05feb362397b7cf7ba13c7719b0a889dd9
Alice First Per Commitment Point: 0368081a0a30ec118e662b71d222c383c5c9f495b4821fd0252452abf659250237
Bob First Per Commitment Secret: 203f2c3e5cd6004ef0ea26e82c2ffeb2dc80036cedca92c210010cd73facd38a
Bob First Per Commitment Point: 03ac685c74c0970c62f0e69ad332b6a5f6d61d3650128ef2a18e011cfd28d177b4


## The Obscured Commitment Number
To more easily allow both sides to keep track of the commitment numbers, each commitment actually encodes the number of the commitment within the lock time and sequence fields of the commitment transaction. Within the protocol, this special encoding is referred to as **state hints**. Assuming a party knows the current commitment number, they’re able to use the state hints to easily recognize if a broadcasted commitment was a revoked one. Rather than encode the state hint in plain sight, an obfuscated state hint is used in its place. This obfuscation is achieved by first XORing the current commitment number with a set of random bytes generated deterministically using the payment basepoint of both sides of the channel. A total of 6 bytes (48 bits) across the [locktime](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction) and [sequence]((https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)) (24 bits of the locktime and 24 bits of the sequence) are used to encode the state hint within the commitment transaction, so 6 random bytes are needed to use for XORing. To obtain these 6 bytes, both sides obtain the SHA-256 hash of the initiator’s payment_basepoint concatenated to the responder’s payment_basepoint.
```
SHA256(payment_basepoint from open_channel || payment_basepoint from accept_channel)
```

In [27]:
# The 48-bit commitment number is obscured by XOR with the lower 48 bits of
# SHA256(payment_basepoint from open_channel || payment_basepoint from accept_channel)

# Commitment number on the opening channel 
commitment_number = 0 

# TODO define the payment_base_point
to_obscure = sha256(bytes.fromhex(alice_payment_basepoint_pubkey) + bytes.fromhex(alice_payment_basepoint_pubkey))
to_obscure_int = int.from_bytes(to_obscure, byteorder='big')
# Extract lower 48 bits
to_obscure_int_lower48 = to_obscure_int & 0xFFFFFFFFFFFF
# obscured commitment number is result of xor operation 
commitment_number_obscured = to_obscure_int_lower48 ^ commitment_number
print("Commitment Number Obscured: " + hex(commitment_number_obscured))

Commitment Number Obscured: 0x62a8021ceabd


## Transaction Inputs
Now we can start creating Alice unsigned commitment transaction. 

In [28]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Calculate the txid of the funding channel tx created on the previous notebook
funding_channel_txid = hash256(unsigned_tx)
# The index of the funding channel tx createt on the previous notebook
funding_channel_txindex = 1
# Convert txid and index to bytes (little endian)
funding_channel_txid = (funding_channel_txid)[::-1]
funding_channel_txindex = funding_channel_txindex.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are upper 24 bits of the obscured commitment number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (1 byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)
print("Inputs: " + inputs.hex())

Inputs: f1cabbb28d85b10a66a39d52d3b775f57e1a76ac5225b05acb7b31b25f13621c01000000008062a802


## Key Derivations

Each commitment transaction uses a unique localpubkey, local_delayedpubkey and revocationpubkey. Those are changed for every transaction based on the per_commitment_point.


The **localdelayed** pubkeys are simply generated by addition from their base points. As definet at Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G
```

The **revocationpubkey** is a blinded key, Alice uses its own `revocation_basepoint` and the Bob node's `first_commitment_point`, sent into `accept_channel` message, to derive the revocationpubkey for the commitment. As definet at Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#revocationpubkey-derivation)):

```
revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
```



## Transaction Outputs

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

As this is the first commitment transaction it will have no outputs to Bob, because Alice was the funder of the channel and she is not sending any Sats to Bob, if she wanted to do that she could use the field `push_msat` on `open_channel` message and define the value in mili satoshis. As the channel was not openned yet, there is no offered or received HTLC, so none HTLC outputs will be created also. So the first Commitment Transaction outputs for Alice and Bob will be the following.

Alice first Commitment Transaction will have two outputs:
* to_local_anchor_output
* to_local_output

Bob first Commitment Transaction will have two outputs:
* to_remote_anchor_output
* to_remote_output

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md))  defines that outputs in transactions are always sorted first according to their value, this way the to_local_anchor_outuput is the first one, as its amount output is fixed at 330 sats, the default dust limit for P2WSH.

**anchor outputs** exists to prevent a malicious peer from attaching child transactions with a low fee density to an anchor and thereby blocking the victim from getting the commit tx confirmed in time. This defense is supported by a change in Bitcoin core 0.19: [bitcoin/bitcoin#15681](https://github.com/bitcoin/bitcoin/pull/15681). This is also the reason that every non-anchor output on the commit tx is CSV locked. The feature is optional and can be enabled if both peers in a channel support it. But [Eclair v0.11.0](https://github.com/ACINQ/eclair/blob/master/docs/release-notes/eclair-v0.11.0.md) stop accepting channels that don't support anchor outputs.

In [32]:
# ALICE DELAYED PUBKEY
# alice_delayed_pubkey = alice_delayed_payment_basepoint + SHA256(per_commitment_point || alice_delayed_payment_basepoint) * G
# alice_delayed_payment_basepoint is generate by alice and sent into the `open_channel` message
# per_commitment_point is generated by alice and sent into the `open_channel` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(alice_first_per_commitment_point) + bytes.fromhex(alice_delayed_payment_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# do the math to get the alice_delayed_pubkey
alice_delayed_pubkey_point = decompress_point(alice_delayed_payment_basepoint_pubkey) + sha_int * G
alice_delayed_pubkey = compress_pubkey(alice_delayed_pubkey_point)
print(f"Alice Delayed Pubkey: {alice_delayed_pubkey }")

# BOB REVOGATION PUBKEY
# revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
# bob_revocation_basepoint is generate by bob and sent into the `accept_channel` message
# per_commitment_point is generated by bob and sent into the `accept_channel` message
# Compute SHA256(revocation_basepoint || per_commitment_point)
sha_output = sha256(bytes.fromhex(bob_revocation_basepoint_pubkey) + bytes.fromhex(bob_first_per_commitment_point))
sha_int = int.from_bytes(sha_output, 'big') % n
# now multiply it by revocation_basepoint
part1 = decompress_point(bob_revocation_basepoint_pubkey) * sha_int
# Compute SHA256(per_commitment_point || revocation_basepoint)
sha_output = sha256(bytes.fromhex(bob_first_per_commitment_point) + bytes.fromhex(bob_revocation_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# multiply it by per_commitment_point
part2 = decompress_point(bob_first_per_commitment_point) * sha_int
# sum the results
bob_revocation_pubkey_point = part1 + part2
bob_revocation_pubkey = compress_pubkey(bob_revocation_pubkey_point)
print(f"Bob Revocation Pubkey: {bob_revocation_pubkey }")

Alice Delayed Pubkey: 03d6363615b3d00361158c0f48a4ef81ea12e214e8d56e24098759f6b4267dca8b
Bob Revocation Pubkey: 03a0596fefba4b53489629115e8b360a95b900f1884306edb98428d5377478b9ac


In [34]:
# OUTPUTS FOR ALICE
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# ANCHOR AMOUNT OUTPUT
anchor_output_value = 330

# ANCHOR OUTPUT  
alice_anchor_output_value = anchor_output_value.to_bytes(8, byteorder="little", signed=True)
# ANCHOR P2WSH
# <local_funding_pubkey/remote_funding_pubkey> OP_CHECKSIG OP_IFDUP
# OP_NOTIF
#     OP_16 OP_CHECKSEQUENCEVERIFY
# OP_ENDIF
to_alice_anchor_redeemScript = bytes.fromhex(
    "21"
    + alice_funding_pubkey
    + "ac"  # OP_CHECKSIG
    + "73"  # OP_IFDUP
    + "64"  # OP_NOTIF
    + "60"  # OP_16
    + "B2"  # OP_CHECKSEQUENCEVERIFY
    + "68") # OP_ENDIF

to_alice_anchor_script_hash = sha256(to_alice_anchor_redeemScript)
to_alice_anchor_output_spk = bytes.fromhex("0020") + to_alice_anchor_script_hash

# TO_LOCAL OUTPUT
# This output sends funds back to the owner of this commitment transaction and thus must be timelocked
# using OP_CHECKSEQUENCEVERIFY. It can be claimed, without delay, by the other party if they know the
# revocation private key. The output is a version-0 P2WSH, with the witness script below:
# OP_IF
    # Penalty transaction
#    <revocationpubkey>
# OP_ELSE
#    `to_self_delay`
#    OP_CHECKSEQUENCEVERIFY
#    OP_DROP
#    <local_delayedpubkey>
# OP_ENDIF
# OP_CHECKSIG
to_alice_redeemScript = bytes.fromhex(
    "63"   # OP_IF
    + "21"
    + bob_revocation_pubkey
    + "67"   # OP_ELSE
    + "02"   
    + "9000" # to_self_delay of 144 blocks
    + "b2"   # OP_CHECKSEQUENCEVERIFY
    + "75"   # OP_DROP
    + "21"
    + alice_delayed_pubkey
    + "68"  # OP_ENDIF
    + "ac") # OP_CHECKSIG

to_alice_script_hash = sha256(to_alice_redeemScript)
to_alice_output_spk = bytes.fromhex("0020") + to_alice_anchor_script_hash



### Fee Calculation

The fee calculation for commitment transactions  is based on the current `feerate_per_kw` sent by Alice into the `open_channel` message and the **expected** weight of the transaction.

The Basis of Lightning Technology [BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#fees) defines the following for the expected weights:
* Commitment weight (no option_anchors):   724 + 172 * num-untrimmed-htlc-outputs
* Commitment weight (option_anchors):     1124 + 172 * num-untrimmed-htlc-outputs
* HTLC-timeout weight (no option_anchors): 663
* HTLC-timeout weight (option_anchors): 666
* HTLC-success weight (no option_anchors): 703
* HTLC-success weight (option_anchors): 706

In [35]:
commitment_weight = 1124

# data from open_channel message
feerate_per_kw = 15000
funding_satoshis = 5000000

commitment_fee_rate = int(commitment_weight * feerate_per_kw / 1000)

# TODO double check if we always subtract two times the anchor value or just one time when there is just one anchor output
alice_output_value = funding_satoshis - anchor_output_value - commitment_fee_rate

to_alice_output_value = alice_output_value.to_bytes(8, byteorder="little", signed=True)

outputs = (
    alice_anchor_output_value
    + varint_len(to_alice_anchor_output_spk)
    + to_alice_anchor_output_spk
    + to_alice_output_value
    + varint_len(to_alice_output_spk)
    + to_alice_output_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_tx: ", unsigned_tx.hex())



unsigned_tx:  0200000001f1cabbb28d85b10a66a39d52d3b775f57e1a76ac5225b05acb7b31b25f13621c01000000008062a802024a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f041a084c000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04201ceabd


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [36]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "a9e10d298bdb129e06c974aa0140034688222ae37415cc80a50d3e50073730bf",
  "hash": "a9e10d298bdb129e06c974aa0140034688222ae37415cc80a50d3e50073730bf",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 3186236448,
  "vin": [
    {
      "txid": "1c62135fb2317bcb5ab02552ac761a7ef575b7d3529da3660ab1858db2bbcaf1",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 44589696
    }
  ],
  "vout": [
    {
      "value": "0.00000330",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 34391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04",
        "desc": "addr(bcrt1qxsu3c7d60r2egf324y2ppayawgqw7408252yzev5z24vg7658uzqwq2zs5)#62txjpcp",
        "hex": "002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04",
        "address": "bcrt1qxsu3c7d60r2egf324y2ppayawgqw7408252yzev5z24vg7658uzqwq2zs5",
        "type": "witness_v0_scripthash"
      }
    },
    {
      "value": "0.04982810"

In [37]:
# value of the funding channel output transaction (Output2)
value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(funding_channel_txid + funding_channel_txindex)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + funding_channel_txid
    + funding_channel_txindex
    + varint_len(redeemScript) # size of scriptcode
    + redeemScript             # scriptcode is the redeemscript of the Output2 (channel funding)
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print("tx_digest_preimage: ",tx_digest_preimage.hex())

tx_digest_preimage:  020000004bb018024043e71ae572dd27d5789a578f01a44667780644a62b43c6100734c3962d31fc29d366db3d2423f79104c6a4d71c11443cd43891d59dedf8ece7a977829c5b655dd991e451b58e44e644704fc33332952bbf10ce71ab2580075724430100000047522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae404b4c00000000008062a802e23c914d5f22c5c348bd6ba3b30b6034d348dd97a03c8dc0a39e8decca9154e7201ceabd01000000


Segwit transactions have a signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)

In [39]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input privkey1
signing_key1 = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
signature1 = signing_key1.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature1 = signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
signing_key2 = ecdsa.SigningKey.from_string(bytes.fromhex(bob_funding_privkey), curve=ecdsa.SECP256k1) 
signature2 = signing_key2.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature2 = signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + varint_len(signature1)
    + signature1
    + varint_len(signature2)
    + signature2
    + varint_len(redeemScript)
    + redeemScript
)

# the final signed transaction
signed_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed commitment transaction: ",signed_commitment_tx.hex())

signed commitment transaction:  02000000000101f1cabbb28d85b10a66a39d52d3b775f57e1a76ac5225b05acb7b31b25f13621c01000000008062a802024a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f041a084c000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f040400483045022100a076020853453214159b01a431dd35bad8dbcda05164d2cddcdac4f9e0105cc0022028cb4d584953c61946b52d28dff4671f7e86e8598d1ca7c9f99452e84062d59001483045022100997eb0eebdab61026e43bfe26925ef1418f6709f3fadb32b4aebc752ae81ea580220772e351fa1ccc8340e5643d6ea8092bf5a4b65c962ed8b01348c6d210d0c07ed0147522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae201ceabd


In [40]:
new_tx_txid = node.sendrawtransaction(signed_tx.hex())
# result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(new_tx_txid)
result = node.testmempoolaccept(rawtxs=[signed_commitment_tx.hex()])
print(result)

829c5b655dd991e451b58e44e644704fc33332952bbf10ce71ab258007572443
[{'txid': 'a9e10d298bdb129e06c974aa0140034688222ae37415cc80a50d3e50073730bf', 'wtxid': '281e610a6fea62e4b9396cb660a83ceef1f73ef72b8cca1232a2a4addb5e5e90', 'allowed': False, 'reject-reason': 'non-final'}]
